In [4]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
import bff
# import pingouin as pg
#import researchpy
# Plotting packages
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

In [5]:
df = pd.read_excel("data60.xlsx")
df

,ResponseId,Country,cond,condName,Intervention_order,Intro_Timer,Intervention_Timer,BELIEFcc,POLICYcc,SHAREcc,...,Trust_sci1_1.1,control_timer2_First Click,control_timer2_Last Click,control_timer2_Page Submit,control_timer2_Click Count,Trust_sci2_1.1,control_timer3_First Click,control_timer3_Last Click,control_timer3_Page Submit,control_timer3_Click Count
0,R_0031kFvKFnFD4E9,Israel,7,PsychDistance,SocialmediasharingPipedText|PolicySupport|Beli...,4.801,104.031,92.75,86.888889,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R_0059fW1EtKYsTWV,Brazil,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,11.155,198.395,100.00,100.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R_00688n4RYG3zIel,Macedonia,10,DynamicNorm,SocialmediasharingPipedText|PolicySupport|Beli...,29.154,83.853,100.00,54.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R_0086kfWbVfurc1H,France,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,8.301,29.399,80.25,85.222222,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R_00B8XyVmrMofqGB,UK,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,17.951,148.918,51.00,48.777778,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54653,R_zZMtNlzas0E8RyN,Slovakia,9,LetterFuture,BeliefinAnthrCC|PolicySupport|Socialmediashari...,31.481,316.675,98.50,91.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54654,R_zZPllZfBJWjfSCZ,Switzerland,9,LetterFuture,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,5.627,206.393,63.00,47.444444,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54655,R_zZPOdDw0XWugcH7,Israel,10,DynamicNorm,PolicySupport|SocialmediasharingPipedText|Beli...,16.698,104.157,98.75,63.888889,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54656,R_zZpwx0yCZNh32ql,Algeria,10,DynamicNorm,BeliefinAnthrCC|PolicySupport|Socialmediashari...,17.577,79.966,13.75,25.750000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Global median split
# df["Ideology"] = (df["Politics2_1"] > df["Politics2_1"].median()).astype(int)

df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2
# Per-country median split
new = []
for _, sub in df.groupby("Country"):
    sub.loc[:, "Ideology"] = (sub.loc[:, "ide"] > sub.loc[:, "ide"].median()).astype(float)
    # Ensure NaN ideology when Politics2_1 is NaN
    sub.loc[sub.Politics2_1.isna(), "Ideology"] = np.NaN
    new.append(sub)
df = pd.concat(new)
# Replace 1 with "Conservative" and 0 with "Liberal"
df['Ideology'] = df['Ideology'].replace(1, 'Conservative').replace(0, 'Liberal')

df_Lib = df.query('Ideology == "Liberal"')
df_Con = df.query('Ideology == "Conservative"')

df_Lib = pd.melt(
    df_Lib.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", "Belief1","Belief2","Belief3","Belief4", "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond", "WEPTcc"],
    var_name="Item",
    value_name="Belief"
)

df_Con = pd.melt(
    df_Con.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", "Belief1","Belief2","Belief3","Belief4", "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond", "WEPTcc"],
    var_name="Item",
    value_name="Belief"
)

df_Lib = df_Lib.dropna()
df_Con = df_Con.dropna()

df_Con['condName'] = df_Con['condName'].replace(['Control'], 'aControl')
df_Lib['condName'] = df_Lib['condName'].replace(['Control'], 'aControl')

In [31]:
df_Lib = df.query('Ideology == "Liberal"')
df_Con = df.query('Ideology == "Conservative"')

df_Lib_p = pd.melt(
    df_Lib.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", 'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9', "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond", "WEPTcc"],
    var_name="Item",
    value_name="Policy"
)

df_Con_p = pd.melt(
    df_Con.loc[:, ['ResponseId', "Country", "condName", "cond", "Ideology","ide", 'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9', "WEPTcc"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond", "WEPTcc"],
    var_name="Item",
    value_name="Policy"
)

df_Lib_p = df_Lib_p.dropna()
df_Con_p = df_Con_p.dropna()

df_Lib_p['condName'] = df_Lib_p['condName'].replace(['Control'], 'aControl')
df_Con_p['condName'] = df_Con_p['condName'].replace(['Control'], 'aControl')


In [32]:
df_Lib_w = df.query('Ideology == "Liberal"')
df_Con_w = df.query('Ideology == "Conservative"')

df_Lib_w = df_Lib_p.dropna()
df_Con_w = df_Con_p.dropna()

df_Lib_w['condName'] = df_Lib_w['condName'].replace(['Control'], 'aControl')
df_Con_w['condName'] = df_Con_w['condName'].replace(['Control'], 'aControl')


# Analyses

In [33]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython
# Load the R library lmerTest
%R library(lmerTest)
%R library(ordinal)
# Send R the dataset
%Rpush df_Lib
%Rpush df_Con

%Rpush df_Lib_p
%Rpush df_Con_p

%Rpush df_Lib_w
%Rpush df_Con_w

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Table S1

In [15]:
%%R

M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Lib)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Lib

REML criterion at convergence: 801747.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.7246 -0.2084  0.0818  0.2655  6.4088 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 425.8121 20.6352 
 Country    (Intercept)  40.5815  6.3704 
 Item       (Intercept)   0.5017  0.7083 
 Residual               129.3631 11.3738 
Number of obs: 95841, groups:  ResponseId, 23981; Country, 57; Item, 4

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 81.7452     1.0416    75.3870  78.481  < 2e-16 ***
condNameBindingMoral         1.3610     0.6729 23908.9969   2.023 0.043133 *  
condNameCollectAction        1.7576     0.6754 23904.8173   2.602 0.009264 ** 
condNameDynamicNorm          0.5148     0.6715 239

# Table S2

In [16]:
%%R

M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Con)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Con

REML criterion at convergence: 786782.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1611 -0.3223  0.0845  0.3315  5.9089 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 563.9459 23.7475 
 Country    (Intercept)  55.6471  7.4597 
 Item       (Intercept)   0.1929  0.4392 
 Residual               153.4700 12.3883 
Number of obs: 91892, groups:  ResponseId, 22997; Country, 57; Item, 4

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 78.1190     1.1687    95.5410  66.843  < 2e-16 ***
condNameBindingMoral         0.6962     0.7848 22926.1485   0.887  0.37498    
condNameCollectAction        2.3673     0.7861 22929.0722   3.011  0.00260 ** 
condNameDynamicNorm          0.3091     0.7864 229

# Table S3

In [24]:
%%R

M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Lib_p)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Lib_p

REML criterion at convergence: 1901147

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8043 -0.5061  0.0695  0.6276  4.9179 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 287.57   16.958  
 Country    (Intercept)  27.03    5.199  
 Item       (Intercept) 154.68   12.437  
 Residual               400.26   20.006  
Number of obs: 209908, groups:  ResponseId, 23886; Country, 57; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 70.9387     4.2240     8.6150  16.794 6.99e-08 ***
condNameBindingMoral         0.7058     0.5755 23731.6334   1.226  0.22010    
condNameCollectAction        3.0374     0.5780 23724.8491   5.255 1.49e-07 ***
condNameDynamicNorm          0.9239     0.5742 2

# Table S4

In [34]:
%%R

M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Con_p)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Con_p

REML criterion at convergence: 1833187

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8467 -0.5543  0.0519  0.6374  4.4159 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 319.45   17.87   
 Country    (Intercept)  41.09    6.41   
 Item       (Intercept) 148.83   12.20   
 Residual               422.77   20.56   
Number of obs: 201072, groups:  ResponseId, 22921; Country, 57; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 69.6680     4.1793     8.9213  16.670 4.98e-08 ***
condNameBindingMoral         0.2290     0.6146 22820.0345   0.373   0.7095    
condNameCollectAction        2.4645     0.6152 22811.4589   4.006 6.20e-05 ***
condNameDynamicNorm          0.4421     0.6157 2

# Table S5


In [28]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName + (1|Country), data = df_Lib_w, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_Lib_w

 link  threshold   nobs   logLik     AIC       niter       max.grad cond.H 
 logit equidistant 209908 -340320.86 680669.71 2366(22657) 1.63e+00 2.0e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1556   0.3944  
Number of groups:  Country 57 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral      0.12872    0.02049   6.281 3.37e-10 ***
condNameCollectAction    -0.06400    0.02034  -3.146  0.00166 ** 
condNameDynamicNorm       0.12297    0.02054   5.988 2.13e-09 ***
condNameFutureSelfCont   -0.01716    0.02157  -0.796  0.42617    
condNameLetterFuture     -0.16158    0.02150  -7.514 5.74e-14 ***
condNameNegativeEmotions -0.25818    0.01975 -13.071  < 2e-16 ***
condNamePluralIgnorance  -0.04592    0.02016  -2.278  0.02274 *  
condNamePsychDistance    -0.19701    0.

# Table S6

In [35]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName + (1|Country), data = df_Con_w, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_Con_w

 link  threshold   nobs   logLik     AIC       niter       max.grad cond.H 
 logit equidistant 201072 -323015.01 646058.03 2347(10566) 1.31e+00 1.9e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2531   0.5031  
Number of groups:  Country 57 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral     -0.05070    0.02076  -2.442  0.01459 *  
condNameCollectAction    -0.15942    0.02067  -7.714 1.22e-14 ***
condNameDynamicNorm      -0.08872    0.02066  -4.293 1.76e-05 ***
condNameFutureSelfCont   -0.22233    0.02170 -10.244  < 2e-16 ***
condNameLetterFuture     -0.32929    0.02191 -15.032  < 2e-16 ***
condNameNegativeEmotions -0.35214    0.02096 -16.799  < 2e-16 ***
condNamePluralIgnorance  -0.11757    0.02061  -5.704 1.17e-08 ***
condNamePsychDistance    -0.41008    0.